[train]_images the image files. Number of images: 21397

train.csv

- image_id the image file name.
- label the ID code for the disease.

sample_submission.csv A properly formatted sample submission, given the disclosed test set content.

- image_id the image file name.
- label the predicted ID code for the disease.
[train/test]_tfrecords the image files in tfrecord format. (not used)

label_num_to_disease_map.json The mapping between each disease code and the real disease name.

code to split the data into two datasets

In [ ]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split

# Set the path to your data root directory
data_root = './data'

# Set the path to the directory for resized images
resized_dir = './resized_data_split_less'

# Set the desired size for the resized images
target_size = (150, 150)

# Set the test size for the train-test split
test_size = 0.2

# Create the directory for resized images if it doesn't exist
os.makedirs(resized_dir, exist_ok=True)

# Iterate through each subdirectory (class)
for illness_class in os.listdir(data_root):
    class_path = os.path.join(data_root, illness_class)
    
    # Skip non-directory files
    if not os.path.isdir(class_path):
        continue

    # Get the list of images in the current class
    images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]

    # Split the images into training and testing sets
    train_images, test_images = train_test_split(images, test_size=test_size, random_state=42)

    # Resize and save each training image to the new directory
    for img in train_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(resized_dir, 'train', illness_class, img)
        os.makedirs(os.path.join(resized_dir, 'train', illness_class), exist_ok=True)

        # Open and resize the image
        with Image.open(src_path) as image:
            resized_image = image.resize(target_size)
            resized_image.save(dest_path)

    # Resize and save each testing image to the new directory
    for img in test_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(resized_dir, 'test', illness_class, img)
        os.makedirs(os.path.join(resized_dir, 'test', illness_class), exist_ok=True)

        # Open and resize the image
        with Image.open(src_path) as image:
            resized_image = image.resize(target_size)
            resized_image.save(dest_path)

print("Images resized and saved to the new directory with train-test split.")


using the code below to format various types of data to test with 

In [ ]:
# Convert and save each training image to the new directory
    for img in train_images:
        src_path = os.path.join(train_class_path, img)
        dest_path = os.path.join(gray_dir, 'train', illness_class, img)

        # Open and convert the image to grayscale
        with Image.open(src_path) as image:
            gray_image = image.convert('L')
            gray_image.save(dest_path)

In [ ]:
import cv2
import numpy as np
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

load all library needed

In [ ]:
#Load libraries
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR
import torchvision
import pathlib
from sklearn.metrics import confusion_matrix

transforms

In [ ]:
transformer=transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),  #0-255 to 0-1, numpy to tensors
    transforms.Normalize([0.5,0.5,0.5], # 0-1 to [-1,1] , formula (x-mean)/std
                        [0.5,0.5,0.5])
])

load the data

In [ ]:
train_path='./resized_data_split/train'
test_path='./resized_data_split/test'

set the batch size for CNN

In [1]:
train_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

NameError: name 'DataLoader' is not defined

simple CNN

In [ ]:
class ConvNet(nn.Module):
    def __init__(self,num_classes=5):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s) +1
        
        #Input shape= (256,3,150,150)
        
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape= (256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape= (256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape= (256,12,150,150)
        
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Reduce the image size be factor 2
        #Shape= (256,12,75,75)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape= (256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape= (256,20,75,75)
        
        self.conv3=nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape= (256,32,75,75)
        self.bn3=nn.BatchNorm2d(num_features=32)
        #Shape= (256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape= (256,32,75,75)
        
        self.fc=nn.Linear(in_features=75 * 75 * 32,out_features=num_classes)
        
        #Feed forwad function
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn3(output)
        output=self.relu3(output)
            
        #Above output will be in matrix form, with shape (256,32,75,75)
            
        output=output.view(-1,32*75*75)
            
        output=self.fc(output)
            
        return output

declare numbner of epoches

In [ ]:
num_epochs=8

set the type of model to be run

In [ ]:
model=ConvNet(num_classes=5).to(device)

set loss function

In [ ]:
loss_function=nn.CrossEntropyLoss()

optimiser to be used, set learning rate and weight decay

In [ ]:
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)

train the model based on above parameters 

In [ ]:
# Initialize the best accuracy variable
best_accuracy = 0.0

# Iterate through each epoch
for epoch in range(num_epochs):
    
    # Set the model to training mode
    model.train()
    
    # Initialize training accuracy and loss
    train_accuracy = 0.0
    train_loss = 0.0
    
    # Iterate through the training dataset
    for i, (images, labels) in enumerate(train_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Zero the gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        
        # Compute the loss
        loss = loss_function(outputs, labels)
        
        # Backward pass
        loss.backward()
        
        # Update the weights
        optimizer.step()
        
        # Update training loss
        train_loss += loss.cpu().data * images.size(0)
        
        # Calculate training accuracy
        _, prediction = torch.max(outputs.data, 1)
        train_accuracy += int(torch.sum(prediction == labels.data))
        
    # Calculate average training accuracy and loss
    train_accuracy = train_accuracy / train_count
    train_loss = train_loss / train_count
    # Initialize the confusion matrix
    conf_matrix = torch.zeros(5, 5)

    # Set the model to evaluation mode
    model.eval()
    
    # Initialize testing accuracy
    test_accuracy = 0.0
    
    # Iterate through the testing dataset
    for i, (images, labels) in enumerate(test_loader_32):
        # Move images and labels to GPU if available
        if torch.cuda.is_available():
            images = Variable(images.cuda())
            labels = Variable(labels.cuda())
            
        # Forward pass
        outputs = model(images)
        
        # Calculate testing accuracy
        _, prediction = torch.max(outputs.data, 1)
        test_accuracy += int(torch.sum(prediction == labels.data))
        # Update the confusion matrix
        conf_matrix += confusion_matrix(labels.cpu().numpy(), prediction.cpu().numpy(), labels=list(range(5)))
    
    # Calculate average testing accuracy
    test_accuracy = test_accuracy / test_count
    class_accuracy = conf_matrix.diag() / conf_matrix.sum(1)

    # Print class-wise accuracy
    for i in range(5):
        print(f'Class {i} Accuracy: {class_accuracy[i].item()}')
        # Print epoch-wise results
    print('Epoch: ' + str(epoch) + ' Train Loss: ' + str(train_loss) +
        ' Train Accuracy: ' + str(train_accuracy) + ' Test Accuracy: ' + str(test_accuracy))
    
    # Save the best model based on testing accuracy
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), 'best_checkpoint.model')
        best_accuracy = test_accuracy

a different model that includes dropout probability 

In [ ]:
class ConvNet_dropout(nn.Module):
    def __init__(self, num_classes=5, dropout_prob=0.5):
        super(ConvNet_dropout, self).__init__()
        
        # Output size after convolution filter ((w-f+2P)/s) + 1
        
        # Input shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape = (256, 12, 150, 150)
        self.relu1 = nn.ReLU()
        # Shape = (256, 12, 150, 150)
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Reduce the image size by factor 2
        # Shape = (256, 12, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 20, 75, 75)
        self.relu2 = nn.ReLU()
        # Shape = (256, 20, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape = (256, 32, 75, 75)
        self.relu3 = nn.ReLU()
        # Shape = (256, 32, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
        output = self.dropout1(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.dropout2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.dropout3(output)
            
        # Above output will be in matrix form, with shape (256, 32, 75, 75)
        output = output.view(-1, 32 * 75 * 75)
            
        output = self.fc(output)
            
        return output


the new model would be set with the code below

In [ ]:
model_dropout = ConvNet_dropout(num_classes=5, dropout_prob=0.1).to(device)
optimizer_dropout=Adam(model_dropout.parameters(),lr=0.001,weight_decay=0.0001)

new data can be loaded for testing like below

In [ ]:
#Path for training and testing directory
train_path='./resized_data_split_less/train'
test_path='./resized_data_split_less/test'

#set the batch size for CNN
train_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform=transformer),
    batch_size=32, shuffle=True
)
test_loader_32=DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform=transformer),
    batch_size=32, shuffle=True
)

model for tanh where relu is replaced by tanh

In [ ]:
class ConvNet_dropout_tanh(nn.Module):
    def __init__(self, num_classes=5, dropout_prob=0.5):
        super(ConvNet_dropout_tanh, self).__init__()
        
        # Output size after convolution filter ((w-f+2P)/s) + 1
        
        # Input shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape = (256, 12, 150, 150)
        self.tanh1 = nn.Tanh()
        # Shape = (256, 12, 150, 150)
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Reduce the image size by factor 2
        # Shape = (256, 12, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 20, 75, 75)
        self.tanh2 = nn.Tanh()
        # Shape = (256, 20, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape = (256, 32, 75, 75)
        self.tanh3 = nn.Tanh()
        # Shape = (256, 32, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.tanh1(output)
            
        output = self.pool(output)
        output = self.dropout1(output)
            
        output = self.conv2(output)
        output = self.tanh2(output)
        output = self.dropout2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.tanh3(output)
        output = self.dropout3(output)
            
        # Above output will be in matrix form, with shape (256, 32, 75, 75)
        output = output.view(-1, 32 * 75 * 75)
            
        output = self.fc(output)
            
        return output


pooling function is set to avg pooling in the new model below

In [ ]:
class ConvNet_dropout_avgpool(nn.Module):
    def __init__(self, num_classes=5, dropout_prob=0.5):
        super(ConvNet_dropout_avgpool, self).__init__()
        
        # Output size after convolution filter ((w-f+2P)/s) + 1
        
        # Input shape = (256, 3, 150, 150)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 12, 150, 150)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        # Shape = (256, 12, 150, 150)
        self.relu1 = nn.ReLU()
        # Shape = (256, 12, 150, 150)
        self.pool = nn.AvgPool2d(kernel_size=2)
        # Reduce the image size by factor 2
        # Shape = (256, 12, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout1 = nn.Dropout(dropout_prob)
        
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 20, 75, 75)
        self.relu2 = nn.ReLU()
        # Shape = (256, 20, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout2 = nn.Dropout(dropout_prob)
        
        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Shape = (256, 32, 75, 75)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        # Shape = (256, 32, 75, 75)
        self.relu3 = nn.ReLU()
        # Shape = (256, 32, 75, 75)
        
        # Dropout layer with specified dropout probability
        self.dropout3 = nn.Dropout(dropout_prob)
        
        self.fc = nn.Linear(in_features=75 * 75 * 32, out_features=num_classes)
        
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
            
        output = self.pool(output)
        output = self.dropout1(output)
            
        output = self.conv2(output)
        output = self.relu2(output)
        output = self.dropout2(output)
            
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        output = self.dropout3(output)
            
        # Above output will be in matrix form, with shape (256, 32, 75, 75)
        output = output.view(-1, 32 * 75 * 75)
            
        output = self.fc(output)
            
        return output
